In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [3]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

/home/igor/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/igor/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [4]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [5]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Part 2 - Now let's make the ANN!

In [ ]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [ ]:
# Initialising the ANN
classifier = Sequential()
# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
# classifier.add(Dropout(rate = 0.1))
# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
# classifier.add(Dropout(rate = 0.1))
# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

# Part 3 - Making predictions and evaluating the model

In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
# Predicting a single new observation
"""Predict if the customer with the following informations will leave the bank:
Geography: France
Credit Score: 600
Gender: Male
Age: 40
Tenure: 3
Balance: 60000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 50000"""
new_prediction = classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction = (new_prediction > 0.5)

In [ ]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [ ]:
cm

# Part 4 - Evaluating, Improving and Tuning the ANN

In [6]:
# Evaluating the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

Using TensorFlow backend.


In [7]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [8]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

In [9]:
mean

0.8298749958351255

In [10]:
variance

0.014235188540961891

In [ ]:
# Improving the ANN
# Dropout Regularization to reduce overfitting if needed


In [11]:
# Tuning the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

In [12]:
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [13]:
classifier = KerasClassifier(build_fn = build_classifier)

In [14]:
parameters = {'batch_size': [20, 25, 30],
              'epochs': [100, 150, 200],
              'optimizer': ['adam', 'rmsprop']}

In [15]:
grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)


In [16]:
grid_search = grid_search.fit(X_train, y_train)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
7200/7200 [==============================] - 1s 134us/step - loss: 0.5384 - acc: 0.7946
Epoch 2/100
7200/7200 [==============================] - 1s 101us/step - loss: 0.4388 - acc: 0.7971
Epoch 3/100
7200/7200 [==============================] - 1s 80us/step - loss: 0.4343 - acc: 0.7971
Epoch 4/100
7200/7200 [==============================] - 1s 97us/step - loss: 0.4316 - acc: 0.7971
Epoch 5/100
7200/7200 [==============================] - 1s 87us/step - loss: 0.4297 - acc: 0.7971
Epoch 6/100
7200/7200 [==============================] - 1s 92us/step - loss: 0.4278 - acc: 0.7971
Epoch 7/100
7200/7200 [==============================] - 1s 91us/step - loss: 0.4290 - acc: 0.7971
Epoch 8/100
7200/7200 [==============================] - 1s 87us/step 

7200/7200 [==============================] - 1s 78us/step - loss: 0.3838 - acc: 0.8413
Epoch 55/100
7200/7200 [==============================] - 1s 76us/step - loss: 0.3860 - acc: 0.8436
Epoch 56/100
7200/7200 [==============================] - 1s 78us/step - loss: 0.3830 - acc: 0.8404
Epoch 57/100
7200/7200 [==============================] - 1s 73us/step - loss: 0.3841 - acc: 0.8429
Epoch 58/100
7200/7200 [==============================] - 1s 92us/step - loss: 0.3817 - acc: 0.8426
Epoch 59/100
7200/7200 [==============================] - 1s 86us/step - loss: 0.3885 - acc: 0.8411
Epoch 60/100
7200/7200 [==============================] - 1s 95us/step - loss: 0.3898 - acc: 0.8383
Epoch 61/100
7200/7200 [==============================] - 1s 108us/step - loss: 0.3855 - acc: 0.8426
Epoch 62/100
7200/7200 [==============================] - 1s 102us/step - loss: 0.3877 - acc: 0.8412
Epoch 63/100
7200/7200 [==============================] - 1s 107us/step - loss: 0.3870 - acc: 0.8397
Epoch 64/1

7200/7200 [==============================] - 0s 68us/step - loss: 0.4214 - acc: 0.8274
Epoch 18/100
7200/7200 [==============================] - 1s 71us/step - loss: 0.4192 - acc: 0.8328
Epoch 19/100
7200/7200 [==============================] - 1s 83us/step - loss: 0.4204 - acc: 0.8313
Epoch 20/100
7200/7200 [==============================] - 1s 80us/step - loss: 0.4192 - acc: 0.8304
Epoch 21/100
7200/7200 [==============================] - 0s 67us/step - loss: 0.4195 - acc: 0.8315
Epoch 22/100
7200/7200 [==============================] - 0s 55us/step - loss: 0.4181 - acc: 0.8315
Epoch 23/100
7200/7200 [==============================] - 0s 49us/step - loss: 0.4204 - acc: 0.8311
Epoch 24/100
7200/7200 [==============================] - 0s 52us/step - loss: 0.4212 - acc: 0.8279
Epoch 25/100
7200/7200 [==============================] - 0s 52us/step - loss: 0.4225 - acc: 0.8311
Epoch 26/100
7200/7200 [==============================] - 0s 53us/step - loss: 0.4213 - acc: 0.8289
Epoch 27/100


7200/7200 [==============================] - 0s 42us/step - loss: 0.3653 - acc: 0.8487
Epoch 81/100
7200/7200 [==============================] - 0s 45us/step - loss: 0.3632 - acc: 0.8504
Epoch 82/100
7200/7200 [==============================] - 0s 40us/step - loss: 0.3659 - acc: 0.8481
Epoch 83/100
7200/7200 [==============================] - 0s 41us/step - loss: 0.3648 - acc: 0.8506
Epoch 84/100
7200/7200 [==============================] - 0s 39us/step - loss: 0.3632 - acc: 0.8490
Epoch 85/100
7200/7200 [==============================] - 0s 39us/step - loss: 0.3639 - acc: 0.8518
Epoch 86/100
7200/7200 [==============================] - 0s 38us/step - loss: 0.3636 - acc: 0.8526
Epoch 87/100
7200/7200 [==============================] - 0s 38us/step - loss: 0.3652 - acc: 0.8526
Epoch 88/100
7200/7200 [==============================] - 0s 39us/step - loss: 0.3634 - acc: 0.8524
Epoch 89/100
7200/7200 [==============================] - 0s 49us/step - loss: 0.3646 - acc: 0.8501
Epoch 90/100


7200/7200 [==============================] - 0s 47us/step - loss: 0.4006 - acc: 0.8376
Epoch 63/100
7200/7200 [==============================] - 0s 42us/step - loss: 0.4007 - acc: 0.8376
Epoch 64/100
7200/7200 [==============================] - 0s 43us/step - loss: 0.3955 - acc: 0.8403
Epoch 65/100
7200/7200 [==============================] - 0s 40us/step - loss: 0.3977 - acc: 0.8372
Epoch 66/100
7200/7200 [==============================] - 0s 42us/step - loss: 0.3966 - acc: 0.8383
Epoch 67/100
7200/7200 [==============================] - 0s 51us/step - loss: 0.3930 - acc: 0.8415
Epoch 68/100
7200/7200 [==============================] - 0s 43us/step - loss: 0.3965 - acc: 0.8353
Epoch 69/100
7200/7200 [==============================] - 0s 43us/step - loss: 0.4003 - acc: 0.8393
Epoch 70/100
7200/7200 [==============================] - 0s 44us/step - loss: 0.3974 - acc: 0.8386
Epoch 71/100
7200/7200 [==============================] - 0s 43us/step - loss: 0.3996 - acc: 0.8372
Epoch 72/100


...

Epoch 45/200
7200/7200 [==============================] - 1s 151us/step - loss: 0.4159 - acc: 0.8343
Epoch 46/200
7200/7200 [==============================] - 1s 151us/step - loss: 0.4187 - acc: 0.8324
Epoch 47/200
7200/7200 [==============================] - 1s 150us/step - loss: 0.4179 - acc: 0.8336
Epoch 48/200
7200/7200 [==============================] - 1s 151us/step - loss: 0.4189 - acc: 0.8354
Epoch 49/200
7200/7200 [==============================] - 1s 158us/step - loss: 0.4161 - acc: 0.8333
Epoch 50/200
7200/7200 [==============================] - 1s 151us/step - loss: 0.4186 - acc: 0.8332
Epoch 51/200
7200/7200 [==============================] - 1s 153us/step - loss: 0.4118 - acc: 0.8336
Epoch 52/200
7200/7200 [==============================] - 1s 151us/step - loss: 0.4181 - acc: 0.8337
Epoch 53/200
7200/7200 [==============================] - 1s 154us/step - loss: 0.4181 - acc: 0.8328
Epoch 54/200
7200/7200 [==============================] - 1s 151us/step - loss: 0.4161 - ac

7200/7200 [==============================] - 1s 152us/step - loss: 0.4369 - acc: 0.7961
Epoch 6/200
7200/7200 [==============================] - 1s 152us/step - loss: 0.4334 - acc: 0.7961
Epoch 7/200
7200/7200 [==============================] - 1s 153us/step - loss: 0.4333 - acc: 0.7961
Epoch 8/200
7200/7200 [==============================] - 1s 151us/step - loss: 0.4312 - acc: 0.7961
Epoch 9/200
7200/7200 [==============================] - 1s 151us/step - loss: 0.4302 - acc: 0.7961
Epoch 10/200
7200/7200 [==============================] - 1s 151us/step - loss: 0.4281 - acc: 0.7961
Epoch 11/200
7200/7200 [==============================] - 1s 151us/step - loss: 0.4248 - acc: 0.8024
Epoch 12/200
7200/7200 [==============================] - 1s 152us/step - loss: 0.4278 - acc: 0.8187
Epoch 13/200
7200/7200 [==============================] - 1s 153us/step - loss: 0.4287 - acc: 0.8212
Epoch 14/200
7200/7200 [==============================] - 1s 151us/step - loss: 0.4273 - acc: 0.8210
Epoch 1

7200/7200 [==============================] - 1s 152us/step - loss: 0.4183 - acc: 0.8297
Epoch 167/200
7200/7200 [==============================] - 1s 152us/step - loss: 0.4223 - acc: 0.8324
Epoch 168/200
7200/7200 [==============================] - 1s 153us/step - loss: 0.4170 - acc: 0.8307
Epoch 169/200
7200/7200 [==============================] - 1s 152us/step - loss: 0.4211 - acc: 0.8328
Epoch 170/200
7200/7200 [==============================] - 1s 151us/step - loss: 0.4183 - acc: 0.8308
Epoch 171/200
7200/7200 [==============================] - 1s 151us/step - loss: 0.4211 - acc: 0.8314
Epoch 172/200
7200/7200 [==============================] - 1s 153us/step - loss: 0.4225 - acc: 0.8311
Epoch 173/200
7200/7200 [==============================] - 1s 152us/step - loss: 0.4224 - acc: 0.8326
Epoch 174/200
7200/7200 [==============================] - 1s 150us/step - loss: 0.4218 - acc: 0.8297
Epoch 175/200
7200/7200 [==============================] - 1s 152us/step - loss: 0.4245 - acc: 0

8000/8000 [==============================] - 1s 185us/step - loss: 0.4204 - acc: 0.8301
Epoch 48/200
8000/8000 [==============================] - 1s 182us/step - loss: 0.4199 - acc: 0.8346
Epoch 49/200
8000/8000 [==============================] - 1s 184us/step - loss: 0.4237 - acc: 0.8309
Epoch 50/200
8000/8000 [==============================] - 1s 181us/step - loss: 0.4227 - acc: 0.8301
Epoch 51/200
8000/8000 [==============================] - 1s 185us/step - loss: 0.4199 - acc: 0.8321
Epoch 52/200
8000/8000 [==============================] - 1s 182us/step - loss: 0.4191 - acc: 0.8305
Epoch 53/200
8000/8000 [==============================] - 1s 183us/step - loss: 0.4219 - acc: 0.8319
Epoch 54/200
8000/8000 [==============================] - 1s 183us/step - loss: 0.4220 - acc: 0.8325
Epoch 55/200
8000/8000 [==============================] - 1s 183us/step - loss: 0.4212 - acc: 0.8322
Epoch 56/200
8000/8000 [==============================] - 1s 182us/step - loss: 0.4228 - acc: 0.8307
Epo

Epoch 128/200
8000/8000 [==============================] - 1s 182us/step - loss: 0.4178 - acc: 0.8342
Epoch 129/200
8000/8000 [==============================] - 1s 185us/step - loss: 0.4205 - acc: 0.8320
Epoch 130/200
8000/8000 [==============================] - 1s 182us/step - loss: 0.4219 - acc: 0.8324
Epoch 131/200
8000/8000 [==============================] - 1s 184us/step - loss: 0.4223 - acc: 0.8340
Epoch 132/200
8000/8000 [==============================] - 1s 183us/step - loss: 0.4209 - acc: 0.8339
Epoch 133/200
8000/8000 [==============================] - 1s 184us/step - loss: 0.4221 - acc: 0.8346
Epoch 134/200
8000/8000 [==============================] - 1s 181us/step - loss: 0.4198 - acc: 0.8316
Epoch 135/200
8000/8000 [==============================] - 1s 184us/step - loss: 0.4222 - acc: 0.8340
Epoch 136/200
8000/8000 [==============================] - 1s 187us/step - loss: 0.4196 - acc: 0.8329
Epoch 137/200
8000/8000 [==============================] - 2s 189us/step - loss: 0

In [17]:
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [18]:
best_parameters

{'batch_size': 25, 'epochs': 200, 'optimizer': 'rmsprop'}

In [19]:
best_accuracy

0.8395